conda install selenium

Add path to chromedriver.exe to your environmental variables

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm_notebook as tqdm
from io import StringIO,BytesIO
from csv import writer
import shutil
from bs4 import BeautifulSoup as bs

import time
import pandas as pd
pd.set_option('display.max_rows', 500)

In [ ]:
# using selenium to navigate the HTML page
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
path_to_chromedriver = 'C:/Users/Desktop/Misc/chromedriver.exe' # change path as needed
browser = webdriver.Chrome(executable_path = path_to_chromedriver, chrome_options=chrome_options)

# go to webportal
url = ''
browser.get(url)

# locate username and password HTML elements
username = browser.find_element_by_id("actionForm_loginId")
password = browser.find_element_by_name("portletInstance_4{actionForm.password}")

# Login with username and password
username.send_keys("")
password.send_keys("")
login_attempt = browser.find_element_by_xpath("//*[@type='image']")
login_attempt.submit()

browser.get('')

# create dataframe with desired columns
addressBook = pd.DataFrame(columns=['?', 'House/Unit/Lot', 'Floor No', 'Building Name', 'Street Type', 
                      'Street Name', 'Postcode', 'Section', 'City', 'Country', 'State'])

# create dataframe for failed records
failed = pd.DataFrame(columns=['street_type','street_name','postcode'])

# read csv containing street type, street names and postcode to be searched
home = pd.read_csv('home_pass_cleaned.csv').drop(columns=['Unnamed: 0'], axis=1)

# locate elements for user input and key in according to input csv file
for i, row in tqdm(home.iloc[99723:].iterrows()):
    hType = browser.find_element_by_id('hType')
    hType.send_keys('Landed')

    streetType = browser.find_element_by_id('actionForm_streetType')
    streetType.send_keys(row.values[0])

    streetName = browser.find_element_by_id('actionForm_streetName')
    streetName.send_keys(row.values[1])

    postcode = browser.find_element_by_id('actionForm_postcode')
    postcode.send_keys(row.values[2])
    
    search = browser.find_element_by_id('submitButton')
    search.submit()
    
    try:
        output = StringIO()
        csv_writer =  writer(output,lineterminator='\n',delimiter ='|')
        
        # user beautiful soup to parse html elements containing the output data
        bs_obj = bs(browser.page_source, 'html.parser')
        
        rows = bs_obj.find_all("table", {"class": "datagrid"})[0].find('tbody').find_all('tr')

        for row in rows:
            cells = row.find_all('td')

            if len(cells) == 0 :
                continue
            
            # collect data in a row
            csv_writer.writerow([cells[1].get_text(),cells[2].get_text(),cells[3].get_text(),
                                  cells[4].get_text(),cells[5].get_text(),cells[6].get_text(),
                                  cells[7].get_text(),cells[8].get_text(),cells[9].get_text(),
                                  cells[10].get_text()]) 

        # write to output file
        with open ('success.csv', 'a') as fd:
            output.seek(0)
            shutil.copyfileobj (output, fd)
    
    # collect failed records
    except:
        with open ('failed.csv','a') as fd:
            fd.write('{}|{}|{}\n'.format(row.values[0],row.values[1],row.values[2]))       
            
    browser.get('')